<a href="https://colab.research.google.com/github/IlyaZutler/GBR-agents/blob/main/GBR_Agents_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import random
import numpy as np
!pip install simpy
import simpy

In [40]:
class Agent:
    def __init__(self, env, location, speed):
        self.env = env
        self.location = location
        self.base_location = location  # Место дислокации
        self.speed = speed
        self.status = 'Free'
        self.total_busy_time = 0
        self.handled_alarms = 0
        self.action = env.process(self.run())

    def run(self):
        while True:
            yield self.env.timeout(1)

    def travel_time(self, destination):
        return abs(self.location - destination) / self.speed

    def travel_to(self, destination):
        while abs(self.location - destination) >= self.speed:
            if self.location < destination:
                self.location = min(self.location + self.speed, destination)
            else:
                self.location = max(self.location - self.speed, destination)
            yield self.env.timeout(1)

    def handle_alarm(self, alarm):
        self.status = 'Busy'
        # Перемещение к тревоге
        yield self.env.process(self.travel_to(alarm.location))
        # Время на тревоге
        yield self.env.timeout(alarm.complexity)
        response_time = self.env.now - alarm.time_created
        self.total_busy_time += response_time
        self.handled_alarms += 1
        self.status = 'Free'  # Изменить статус на Free сразу после обработки тревоги
        alarm.response_time = response_time

        # Возвращение к месту дислокации
        yield self.env.process(self.travel_to(self.base_location))

        # Удаление тревоги из списка после обработки
        if alarm in self.env.simulation.alarms:
            self.env.simulation.alarms.remove(alarm)

class Alarm:
    def __init__(self, env, location, complexity):
        self.env = env
        self.location = location
        self.complexity = complexity
        self.time_created = env.now
        self.response_time = 0

class Simulation:
    def __init__(self, num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration):
        self.env = simpy.Environment()

        self.segment_length = segment_length
        self.agents = [
            Agent(self.env, location=segment_length / num_agents / 2 + i * (segment_length / num_agents), speed=agent_speed)
            for i in range(num_agents)
        ]
        self.alarm_rate = alarm_rate
        self.complexity_mean = complexity_mean
        self.complexity_sigma = complexity_sigma
        self.threshold = threshold
        self.simulation_duration = simulation_duration
        self.alarms = []
        self.total_alarms = 0
        self.total_response_time = 0
        self.delayed_alarms = 0
        self.results = {'total_alarms': 0, 'average_response_time': 0, 'alarms_above_threshold': 0, 'busy_agents': []}

    def generate_alarms(self):
        while True:
            yield self.env.timeout(random.expovariate(self.alarm_rate))
            location = random.uniform(0, self.segment_length)
            complexity = np.random.lognormal(mean=self.complexity_mean, sigma=self.complexity_sigma)
            alarm = Alarm(self.env, location, complexity)
            self.alarms.append(alarm)
            self.dispatch_agent(alarm)

    def dispatch_agent(self, alarm):
        free_agents = [agent for agent in self.agents if agent.status == 'Free']
        if free_agents:
            nearest_agent = min(free_agents, key=lambda agent: agent.travel_time(alarm.location))
            self.env.process(nearest_agent.handle_alarm(alarm))
        else:
            print(f"No available agents at time {self.env.now}")

    def monitor(self):
        while True:
            yield self.env.timeout(1)
            busy_agents = sum(agent.status == 'Busy' for agent in self.agents)
            self.results['busy_agents'].append(busy_agents / len(self.agents))

    def report(self):
        self.total_alarms = len(self.alarms)
        self.total_response_time = sum(alarm.response_time for alarm in self.alarms)
        self.delayed_alarms = sum(1 for alarm in self.alarms if alarm.response_time > self.threshold)
        average_response_time = self.total_response_time / self.total_alarms if self.total_alarms > 0 else 0
        average_busy_agents = np.mean(self.results['busy_agents']) if self.results['busy_agents'] else 0
        average_delay_ratio = self.delayed_alarms / self.total_alarms if self.total_alarms > 0 else 0

        print("Total alarms:", self.total_alarms)
        print("Average response time:", average_response_time)
        print("Alarms above threshold:", self.delayed_alarms)
        print("Average delay ratio:", average_delay_ratio)
        print("Busy agents over time:", average_busy_agents)
        for i, agent in enumerate(self.agents):
            print(f"Agent {i} handled {agent.handled_alarms} alarms, busy time: {agent.total_busy_time}")

    def run(self):
        self.env.process(self.generate_alarms())
        self.env.process(self.monitor())
        self.env.run(until=self.simulation_duration)
        self.report()

if __name__ == "__main__":
    # Настраиваемые параметры
    num_agents = 10
    segment_length = 1.0
    agent_speed = 0.1
    alarm_rate = 1 / 10  # Средний интервал между тревогами
    complexity_mean = 0
    complexity_sigma = 1
    threshold = 5
    simulation_duration = 100  # Время моделирования

    random.seed(42)
    np.random.seed(42)

    simulation = Simulation(num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration)
    simulation.run()


AttributeError: 'Environment' object has no attribute 'simulation'